# Sound Classification with CatBoost and Hyperparameter Tuning

In [1]:
import os
import numpy as np
import pandas as pd
import librosa as lb
import IPython.display as ipd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm

In [2]:
metadata = pd.read_csv('C:/Users/user/PycharmProjects/pythonProject/CatSound/metadata/cat2.csv')
print(metadata.shape)
metadata.head()

(1120, 4)


,slice_file_name,fold,classID,class
0,purring (1).wav,1,0,purring
1,purring (2).wav,1,0,purring
2,purring (3).wav,1,0,purring
3,purring (4).wav,1,0,purring
4,purring (5).wav,1,0,purring


In [3]:
classes = metadata.groupby('classID')['class'].unique()
classes

classID
0     [purring]
1        [meow]
2    [grooming]
3    [drinking]
4          [wc]
5      [eating]
Name: class, dtype: object

In [4]:
import librosa
import librosa.display

In [5]:
def feature_extractor(path):
    data, simple_rate = librosa.load(path)
    data = librosa.feature.mfcc(data,n_mfcc=128,sr=simple_rate)
    data = np.mean(data,axis=1)
    return data

In [ ]:
#feature extraction with mfcc
x, y = [], [] 
extracted_features=[]
for i,rows in tqdm(metadata.iterrows()):
    path = 'C:/Users/user/PycharmProjects/pythonProject/CatSound/audio/' + 'fold' + str(rows['fold']) + '/' + str(rows['slice_file_name'])
    x.append(feature_extractor(path))
    y.append(rows['classID'])
    fe = feature_extractor(path)
    final_class_labels=rows['class']
    extracted_features.append([fe,final_class_labels])
x = np.array(x)
y = np.array(y)
x.shape, y.shape

In [7]:
#each audio has mfcc matrix
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-312.18478, 125.071014, -64.47501, 56.468628,...",purring
1,"[-408.58997, 146.62447, 32.391697, 36.671593, ...",purring
2,"[-440.81326, 190.86948, -21.095238, -6.7931995...",purring
3,"[-373.53268, 150.26164, 44.241188, 31.615002, ...",purring
4,"[-578.2753, 163.7805, 61.469208, 17.083605, -3...",purring


In [8]:
xtrainval, xtest, ytrainval, ytest = train_test_split(x,y,test_size=0.1,stratify=y,random_state=387)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrainval,ytrainval,test_size=0.2,stratify=ytrainval,random_state=387)
print('\nNumber of samples for Train set :',xtrain.shape[0])
print('Number of samples for Validation set :',xvalid.shape[0])
print('Number of samples for Test set :',xtest.shape[0])


Number of samples for Train set : 806
Number of samples for Validation set : 202
Number of samples for Test set : 112


In [9]:
xtrain.shape

(806, 128)

In [10]:
from catboost import CatBoostClassifier
from catboost import CatBoost
from catboost import Pool
from catboost import cv
from catboost.utils import eval_metric
from catboost import Pool
import catboost as cb

In [12]:
model = cb.CatBoostClassifier(loss_function='MultiClass', 
                              eval_metric='Accuracy')

In [13]:
grid = {'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5,],
        'iterations':[5,10,50],
        'learning_rate':[0.01, 0.03, 0.1,1.0],
        'bootstrap_type':['Bayesian', 'Bernoulli', 'MVS', 'No']}

In [15]:
train_data = Pool(xtrain, ytrain)
eval_data = Pool(xvalid, yvalid)

In [ ]:
model.grid_search(grid,train_data,plot=True)

In [17]:
history = model.fit(train_data, 
          eval_set=eval_data,
          early_stopping_rounds=50,
          plot=True,
          silent=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8163772	test: 0.7722772	best: 0.7722772 (0)	total: 37ms	remaining: 1.81s
1:	learn: 0.8883375	test: 0.8019802	best: 0.8019802 (1)	total: 69.3ms	remaining: 1.66s
2:	learn: 0.9193548	test: 0.8613861	best: 0.8613861 (2)	total: 104ms	remaining: 1.62s
3:	learn: 0.9478908	test: 0.8861386	best: 0.8861386 (3)	total: 137ms	remaining: 1.57s
4:	learn: 0.9677419	test: 0.9059406	best: 0.9059406 (4)	total: 169ms	remaining: 1.52s
5:	learn: 0.9801489	test: 0.8861386	best: 0.9059406 (4)	total: 202ms	remaining: 1.48s
6:	learn: 0.9950372	test: 0.9009901	best: 0.9059406 (4)	total: 236ms	remaining: 1.45s
7:	learn: 0.9962779	test: 0.9306931	best: 0.9306931 (7)	total: 268ms	remaining: 1.41s
8:	learn: 0.9987593	test: 0.9405941	best: 0.9405941 (8)	total: 301ms	remaining: 1.37s
9:	learn: 0.9987593	test: 0.9455446	best: 0.9455446 (9)	total: 334ms	remaining: 1.33s
10:	learn: 1.0000000	test: 0.9554455	best: 0.9554455 (10)	total: 367ms	remaining: 1.3s
11:	learn: 1.0000000	test: 0.9504950	best: 0.9554455 

In [27]:
print("\n The best score across ALL searched params:\n", model.best_score_)


 The best score across ALL searched params:
 {'learn': {'Accuracy': 1.0, 'MultiClass': 0.002529426432783263}, 'validation': {'Accuracy': 0.9851485148514851, 'MultiClass': 0.05290394583637222}}


In [29]:
from sklearn.metrics import confusion_matrix

In [39]:
#Confusion matrix to calculate accuracy for each class
ytrue = (ytest)
ypred = (model.predict(xtest))
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(ytrue,ypred))
print('\n\nClassification Report : \n\n',classification_report(ytrue,ypred))


Confusion Matrix :


[[19  0  1  0  0  0]
 [ 0 19  0  0  0  1]
 [ 0  0 20  0  0  0]
 [ 0  0  0 19  0  1]
 [ 0  0  0  0 11  1]
 [ 0  0  0  0  0 20]]


Classification Report : 

               precision    recall  f1-score   support

           0       1.00      0.95      0.97        20
           1       1.00      0.95      0.97        20
           2       0.95      1.00      0.98        20
           3       1.00      0.95      0.97        20
           4       1.00      0.92      0.96        12
           5       0.87      1.00      0.93        20

    accuracy                           0.96       112
   macro avg       0.97      0.96      0.96       112
weighted avg       0.97      0.96      0.96       112



In [75]:
model.save_model("model.py")

In [53]:
model.get_params()

{'loss_function': 'MultiClass',
 'eval_metric': 'Accuracy',
 'depth': 6,
 'l2_leaf_reg': 1,
 'iterations': 50,
 'learning_rate': 1.0,
 'bootstrap_type': 'MVS'}